# pycoalescence fragmented landscape example

The process outlined here simulates a community of 18010 individuals on a real landscape as it progresses towards a randomly-cleared landscape of 2495 individuals over 100 generations. The community is sampled every 10 generations. The simulation is repeated 10 times and the community is generated for three different speciation rates.

### Inputs

- The following map files in the "maps" folder
	- "fine_present.tif"
	- "coarse_present.tif"
	- "fine_historical.tif"
	- "coarse_historical.tif"
- A speciation rates of 0.0001 (the minimum), 0.0005 and 0.001
- A dispersal sigma of 16
- A deme size of 10

### Outputs

- 10 databases (one per simulation) in the "output" folder, containing the following features in the respective SQL table:
	- The simulation parameters (SIMULATION_PARAMETERS)
	- The community parameters (COMMUNITY_PARAMETERS)
	- The full coalescence tree (SPECIES_LIST)
	- The species abundances for each community (SPECIES_ABUNDANCES)
- A csv containing the species richness at each time
- A csv containing the species abundances 0, 50 and 100 generations before present

In [1]:
from pycoalescence import Simulation, CoalescenceTree
import pandas as pd
import os
import time

## Running the simulation and generating the communities

The `Simulation` class performs the simulation. The `CoalescenceTree` class calculates the coalescence tree for each parameter set and generates the community.

In [2]:
# Run 10 repeat simulations, each with a different random number seed
start_time = time.time()
for seed in range(1, 11, 1):
    # Set up and run the simulation
    sim = Simulation(logging_level=30)
    sim.set_simulation_parameters(seed=seed, task=1, output_directory="output",
                                  min_speciation_rate=0.000001, sigma=16, deme=5)
    if not os.path.exists(os.path.join("output", "data_{}_{}.db".format(1, seed))):
        sim.set_map_files(sample_file="null", 
                          fine_file=os.path.join("maps", "fine_present.tif"),
                          coarse_file=os.path.join("maps", "coarse_present.tif"))
#         sim.set_map_files(sample_file="null", fine_file=os.path.join("maps", "fine_historical.tif"),
#                                coarse_file=os.path.join("maps", "coarse_historical.tif"))
        sim.add_historical_map(fine_file=os.path.join("maps", "fine_historical.tif"),
                               coarse_file=os.path.join("maps", "coarse_historical.tif"), time=99.9)
        sim.add_sample_time([x for x in range(0, 160, 10)])
        # This is the main time-consuming step
        sim.run()
    print("Sim {} complete.".format(seed))
    # Now generate the coalescence tree for each speciation rate and time
    coal_tree = CoalescenceTree(sim)
    coal_tree.set_speciation_parameters(speciation_rates=[0.000001, 0.000005, 0.00001],
                                        times=[float(x) for x in range(0, 160, 10)])
    coal_tree.apply()
    # Calculate the species richness for each parameter and store it in a new table
    coal_tree.calculate_richness()
end_time = time.time()
print("Total time: {} seconds".format(end_time - start_time))


Sim 1 complete.
Sim 2 complete.
Sim 3 complete.
Sim 4 complete.
Sim 5 complete.
Sim 6 complete.
Sim 7 complete.
Sim 8 complete.
Sim 9 complete.
Sim 10 complete.
Total time: 624.9451930522919 seconds


## Store the species richness at each time

In [3]:
output = []
for seed in range(1, 11, 1):
	coal_tree = CoalescenceTree(os.path.join("output", "data_{}_{}.db".format(1, seed)))
	for ref in coal_tree.get_community_references():
		time = coal_tree.get_community_parameters(ref)["time"]
		sr = coal_tree.get_species_richness(ref)
		spec_rate = coal_tree.get_community_parameters(ref)["speciation_rate"]
		output.append({"seed" : seed, "time" : time, "species_richness" : sr, "speciation_rate" : spec_rate})
output_df = pd.DataFrame(output)
output_df.to_csv(os.path.join("results", "richness.csv"))

## Store the species abundances at select times

Just store the time for 0, 50 and 100 generations before the present. 

In [4]:
output = []
for seed in range(1, 11, 1):
	coal_tree = CoalescenceTree(os.path.join("output", "data_{}_{}.db".format(1, seed)))
	for ref in coal_tree.get_community_references():
		time = coal_tree.get_community_parameters(ref)["time"]
		if time in [0, 50, 100, 150]:
			spec_abun = coal_tree.get_species_abundances(reference=ref)
			speciation_rate = coal_tree.get_community_parameters(ref)["speciation_rate"]
			for row in spec_abun:
				output.append({"time" : time, "species_id" : row[0], "abundance" : row[1], "seed" : seed,
							   "speciation_rate" : speciation_rate})
output_df = pd.DataFrame(output)
output_df.to_csv(os.path.join("results", "species_abundances.csv"))